In [2]:
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np


import logging

import dvc.api


import patsy as ps
import os, sys
from IPython.display import Image, display

In [3]:
Format = '%(levelname)s %(asctime)s - %(message)s'
logging.basicConfig(filename= '../notebook/causal.log',
level =logging.INFO,
format = Format)
logger = logging.getLogger()

In [4]:
#Get url from DVC

path = r'C:\Users\Jakinda\Documents\Python Scripts\10Academy\causalinference\data\data.csv'
repo = r'C:\Users\Jakinda\Documents\Python Scripts\10Academy\causalinference'
version = 'v1'

data_url2 = dvc.api.get_url(

    path = path,
    repo = repo,
    rev = version

)
logger.info('Loading data from dvc')

In [5]:
data = pd.read_csv(data_url2,sep=',')
data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [6]:
data = data.drop('Unnamed: 32', axis =1)

In [7]:
# Creating a list of columns with only the columns that represent the mean.
mean_columns = data[[ 'radius_mean', 'texture_mean', 'perimeter_mean',
             'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
             'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']]
# Creating a list of columns with only the columns that represent the standard error .             
se_columns = data[['radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se']]
# Creating a list of columns with only the columns that represent the worst values.
worst_columns = data[['diagnosis','radius_worst', 'texture_worst',
              'perimeter_worst', 'area_worst', 'smoothness_worst',
              'compactness_worst', 'concavity_worst', 'concave points_worst',
              'symmetry_worst', 'fractal_dimension_worst']]

In [8]:
all_dummies = mean_columns+se_columns+worst_columns

In [9]:
model = CausalModel(
    data=data,
    treatment=['radius_mean'], 
    outcome=['diagnosis'], 
    common_causes=all_dummies, 
    #instruments=['iv_obstruction', 'iv_protection'], 
)
model.view_model()
display(Image(filename="causal_model.png"))

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().